<a href="https://colab.research.google.com/github/AlexKressner/Business_Intelligence/blob/main/openai_api_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installation
! pip install -q --upgrade cohere typing-extensions==4.5.0 openai tiktoken python-dotenv beautifulsoup4

In [ ]:
from openai import OpenAI
import tiktoken
import math
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

# OpenAI API Keys
Damit Sie die OpenAI API nutzen können, müssen Sie einen API Key erstellen. Mit diesem authentifizieren Sie sich, wenn Sie Anfragen an den Service stellen. Ihren **geheimen** API Key können Sie (u.a.) auf zwei Arten speichern:

1. Erstellen Sie ein sogenanntes `.env`-file und speichern Sie Ihren API Key darin (`OPENAI_API_KEY={API Key}`). Aus diesem wird der Key als Umgebungsvariable mittels `dotenv` geladen. Das `.env`-file wird gelöscht, sobald die Laufzeitumgebung in Colab von Ihnen geöscht wird.

2. Fügen Sie Ihren API Key direkt in eine Notebookzelle ein. ACHTUNG: Das ist definitiv kein Best-Practice. Würden Sie Ihre Colab-Datei auf github veröffentlichen, würde der API Key vermutlich recht schnell geklaut werden! Da Sie Ihre Notebook aber nirgends veröffentlichen, ist dies auch eine Option.

Einen API Key können Sie [hier](https://platform.openai.com/api-keys) erstellen.

In [ ]:
# Variante 1
secret_key = "{API Key}"

In [ ]:
# Variante 2
# Lädt die Umgebungsvariablen aus der .env-Datei
load_dotenv()

# Zugriff auf die Umgebungsvariablen
secret_key = os.getenv('OPENAI_API_KEY')

# Einstieg
Eine ausführliche Dokumentation zur OpenAI Entwickler-Plattform finden Sie unter diesem [Link](https://platform.openai.com/docs/overview). Die genaue Spezifikation der API, die wir verwenden werden, können Sie unter folgendem [Link](https://platform.openai.com/docs/api-reference) nachschlagen.

Um Anfragen über die OpenAI API zu senden, müssen Sie zunächst ein Client-Objekt erstellen (`client = OpenAI()`). Den API Key übergeben wir als Argument.

In [ ]:
client = OpenAI(api_key=secret_key)

Bei Verwendung der Chat-API unterscheidet man zwischen drei Rollen: **System**, **Assistent** und **Nutzer**.

Beim **Assistenten** handelt es sich um das eigentliche Chat-Modell wie z.B. `gpt-3.5` oder `gpt-4`. Dieses tritt im Rahmen eines Chats (Konversation) in Interaktion mit dem **Nutzer**. Der Nutzer sendet einen Prompt an den Assistenten und dieser antwortet darauf. Das grundlegende Verhalten des Assistenten wird über die Beschreibung des **System**s gestaltet.

In [ ]:
default_system_message = "Du bist ein freundlicher und hilfsbereiter Assistent, der Kunden bei Fragen hilft"

In [ ]:
# Hilfsfunktion zur Interaktion mit der Chat-API
def get_completion(prompt, system_message=default_system_message, model="gpt-3.5-turbo"):
    system_message = [{"role": "system", "content": system_message}] # Wie soll sich das System grundlegend verhalten
    messages = [{"role": "user", "content": prompt}] # Prompt des Nutzers
    response = client.chat.completions.create(
        model=model,
        messages=system_message+ messages,
    )
    return response.choices[0].message.content

In [ ]:
prompt = "Ich habe ein Problem mit meinem Smartphone und benötige Hilfe!"

In [ ]:
get_completion(prompt)

In [ ]:
system_message = "Du bist ein eher mürrischer Assistent. Du sprichst möglich wenig Wörter"

In [ ]:
get_completion(prompt, system_message)

# Der Chat-Endpunkt im Detail
Der Chat-Endpunkt ist detailliert unter dem folgendem [Link](https://platform.openai.com/docs/api-reference/chat?lang=python) dokumentiert. Wir betrachten zunächst das "chat completion object", d.h. die Antwort des Chat-Endpunkts.

In [ ]:
prompt = "Hallo, ich bin Alex"

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
    ],
    )
response

In [ ]:
# Attribute des ChatCompletion Objekts
response.choices[0].message.content, response.usage

# Tokens
"Token" im Zusammenhang mit Large Language Models (LLMs) wie GPT-3 oder GPT-4 beziehen sich auf die grundlegenden Einheiten der Datenverarbeitung, die das Modell verwendet, um Text zu interpretieren und zu generieren. Ein Token kann ein Wort, ein Teil eines Wortes oder sogar ein einzelnes Zeichen sein. Ein Text wird in die definierten Tokens zerlegt. Anschließend wir jeder Token in einen numerischen Wert umgewandelt. Diese Werte sind Indizes, die den entsprechenden Token im Vokabular des Modells repräsentieren.

Wir verwenden hier den Byte Pair Encoding (BPE) Tokenizer [tiktoken](https://github.com/openai/tiktoken), der speziell für die Nutzung mit OpenAI Modellen entwickelt wurde. Ein BPE Tokenizer teilt Wörter in häufig vorkommende Buchstabenpaare oder Gruppen auf. Es ist besonders nützlich, um mit einem begrenzten Vokabular eine Vielzahl von Wörtern und Wortformen abzudecken.

Ein ausführliches Beispiel zur Verwendung von tiktoken finden Sie [hier](https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken)

In [ ]:
prompt = "Hallo, meine Name ist Alexander Kressner"

In [ ]:
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

In [ ]:
tokens = encoding.encode(prompt)
tokens

In [ ]:
encoding.decode(tokens)

In [ ]:
[encoding.decode_single_token_bytes(token) for token in tokens]

In [ ]:
name = "Alexander"
prompt = f"Bitte buchstabiere meinen Namen '{name}' rückwärts"

In [ ]:
encoding.encode(name)

In [ ]:
get_completion(prompt)

In [ ]:
name = "A-l-e-x-a-n-d-e-r"
prompt = f"Bitte buchstabiere meinen Namen '{name}' rückwärts"

In [ ]:
tokens = encoding.encode(name)
tokens

In [ ]:
[encoding.decode_single_token_bytes(token) for token in tokens]

In [ ]:
get_completion(prompt)

**FRAGE**
Wie viele Token hat der Satz `"Ich studiere an der Dualen Hochschule Baden-Württemberg!"`

## Argumente für den Chat-Endpunkt
Wenn Sie Anfragen an den [Chat-Endpunkt](https://platform.openai.com/docs/api-reference/chat) stellen, können Sie verschiedene Argumente übergeben. Die wichtigsten wollen wir einmal näher betrachten.

In [ ]:
prompt = "Hallo, meine Name ist Alex"

In [ ]:
default_keyword_args = {
    "model":"gpt-3.5-turbo",
    "messages":[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
    ],
}

### `logprobs`
Wenn diese Option aktiviert ist (also "true"), dann gibt das System die logarithmierten Wahrscheinlichkeiten jedes einzelnen Ausgabetokens, die im Inhalt der Nachricht enthalten sind, zurück.

In [ ]:
keyword_args = default_keyword_args | {"logprobs":True}

In [ ]:
response = client.chat.completions.create(**keyword_args)
response.choices[0].message.content

In [ ]:
response.choices[0]

In [ ]:
math.exp(response.choices[0].logprobs.content[0].logprob)

In [ ]:
{Token.token:math.exp(Token.logprob) for Token in response.choices[0].logprobs.content}

### `top_logprobs`
Beschreibt einen Parameter, der einen ganzzahligen Wert zwischen 0 und 5 annimmt. Dieser Wert spezifiziert die Anzahl der wahrscheinlichsten Tokens, die an jeder Token-Position zurückgegeben werden sollen, wobei jedem Token eine zugehörige logarithmierte Wahrscheinlichkeit beigefügt ist. Der Parameter `logprobs` muss auf `true` gesetzt werden.

In [ ]:
keyword_args = default_keyword_args | {"logprobs":True, "top_logprobs":3}

In [ ]:
response = client.chat.completions.create(**keyword_args)
response.choices[0].message.content

In [ ]:
response.choices[0]

In [ ]:
toplogs = {TopLog.token:math.exp(TopLog.logprob) for TopLog in response.choices[0].logprobs.content[0].top_logprobs}

In [ ]:
toplogs

### `logit_bias`
Ändere die Wahrscheinlichkeit des Erscheinens bestimmter Tokens in der Vervollständigung. Die Bias-Werte reichen von -100 bis 100. Negative/Positive Werte verringern/erhöhen die Wahrscheinlichkeit der Auswahl. Werte von -100 oder 100 sollten zu einem Verbot des relevanten Tokens führen.

In [ ]:
token = encoding.encode("Hallo")
token

In [ ]:
keyword_args = default_keyword_args | {"logit_bias":{token[0]:-100}}

In [ ]:
response = client.chat.completions.create(**keyword_args)
response.choices[0].message.content

**FRAGE**

1) Wie können Sie weitere Begrüßungswörter verhindern?

2) Wie stellen Sie sicher, dass ein Wort in der Antwort des Assistenten erscheint?

### `max_tokens`
Die maximale Anzahl an Tokens, die in der Chat-Vervollständigung generiert werden können. Die Gesamtlänge der Eingabetokens und der generierten Tokens wird durch die Kontextlänge des Modells begrenzt.

In [ ]:
print(f"""
      Anzahl input token gesamt = {response.usage.prompt_tokens} \n
      Anzahl output token gesamt = {response.usage.completion_tokens} \n
      Anzahl token gesamt = {response.usage.total_tokens}
""")

In [ ]:
keyword_args = default_keyword_args | {"max_tokens":10}

In [ ]:
response = client.chat.completions.create(**keyword_args)
response.choices[0].message.content

### `response_format`
Spezifiziert das Ausgabeformat des Modells (JSON oder Text).

In [ ]:
import json

In [ ]:
keyword_args = default_keyword_args | {
    "response_format":{"type": "json_object"},
    "model":"gpt-3.5-turbo-1106",
    "messages":[
    {"role": "system", "content": "You are a helpful assistant and answer in json format."},
    {"role": "user", "content": prompt}
    ]
}


In [ ]:
response = client.chat.completions.create(**keyword_args)

In [ ]:
response_dict = json.loads(response.choices[0].message.content)

In [ ]:
response_dict["response"]

### `temperatur`
Der Parameter kann Werte zwischen 0 und 2 annehmen. Höhere Werte wie 0,8 machen die Ausgabe zufälliger, während niedrigere Werte wie 0,2 sie fokussierter und deterministischer machen.

In [ ]:
prompt = "Bitte schreib mir ein Gedicht, dass Studierende motiviert der Vorlesung zur künstlichen Intelligenz aufmerksam zu folgen und aktiv teilzunehmen! Bitte schreibe maximal 30 Wörter!"

In [ ]:
default_keyword_args = {
    "model":"gpt-3.5-turbo",
    "messages":[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
    ],
}

In [ ]:
keyword_args = default_keyword_args | {"temperature":0}

In [ ]:
response = client.chat.completions.create(**keyword_args)
print(response.choices[0].message.content)

In [ ]:
keyword_args = default_keyword_args | {"temperature":1.5}

In [ ]:
response = client.chat.completions.create(**keyword_args)
print(response.choices[0].message.content)

# Beispiel: Web-Scapping + Chat API
Wir wollen in nachfolgenden Beispiel Daten von einer Website laden (scrappen) und mittels Chat-API analysieren. Wir nutzen dafür die Bibliothek `requests` und `BeautifulSoup`. Das Python-Paket `requests` ist eine benutzerfreundliche Bibliothek, die das Senden von HTTP-Anfragen ermöglicht und eine einfache Verwendung für den Zugriff auf Web-Ressourcen bietet. `BeautifulSoup` ist eine Python-Bibliothek, die zur Analyse und Extraktion von Daten aus HTML- und XML-Dateien dient und dabei eine einfache Schnittstelle für das Parsen und Navigieren im Dokumentenbaum bietet.

In [ ]:
def get_website_text(url: str):
    response = requests.get(url)

    # Überprüfen, ob die Anfrage erfolgreich war
    if response.status_code == 200:
        # Parsen des HTML-Inhalts der Seite
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extrahieren sämtlichen Textes der Webseite
        text = soup.get_text(separator=' ', strip=True)
        return text
    else:
        # Zurückgeben einer Fehlermeldung mit dem HTTP-Statuscode
        return f"Fehler beim Abrufen der Webseite: HTTP-Statuscode {response.status_code}"

In [ ]:
text = get_website_text("https://www.lappcareer.com/stellenangebote.html?")

In [ ]:
print(text)

In [ ]:
def get_completion(prompt, system_message=default_system_message, model="gpt-3.5-turbo"):
    system_message = [{"role": "system", "content": system_message}]
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=system_message+ messages,
    )
    return response.choices[0].message.content

In [ ]:
prompt = f"""
Du erhälst nachfolgend einen Text, der die ausgeschriebenen Stellen eines Unternehmens auf seiner Website wiedergibt!
Gibt es Stellenausschreibungen für ein Duales Studium? Falls ja, bitte nenne die Ausschreibungen!
Der relevante Text folgt nach dem Doppelpunkt: {text}
"""

In [ ]:
print(get_completion(prompt))

**Frage**
Wie gut funktioniert die Suche nach anderen Stellenausschreibungen, wenn Sie ein anderen "Such-Prompt" verwenden?

# Aufgabe: Witzemaschine
Sie alle kennen schlechten Wortwitze wie zum Beispiel: `Was ist grün und klopft an die Tür?` - Antwort: `Ein Klopfsalat!`. Versuchen Sie mit Hilfe von Few-Shot-Prompting eine Witze-Maschine zu programmieren, die auf eine Frage eines Nutzers mit einem Wortwitz antwortet!

In [ ]:
default_system_message = "Du bist ein KI-System, das Wortwitze erstellt. Du antwortest dabei bitte in einer konsistenten Art und Weise."

In [ ]:
Frage =[
    "<Frage>: ...?",
    "<Frage>: ...?",
]

In [ ]:
Antwort =[
    "<Antwort>: ...!",
    "<Antwort>: ...!",
]

In [ ]:
examples = []
for example in range(len(Frage)):
  examples += [{"role": "user", "content": Frage[example]}] + [{"role": "assistant", "content": Antwort[example]}]


In [ ]:
prompt = "<Frage>: ...?"

In [ ]:
def get_completion(prompt, system_message=default_system_message, model="gpt-3.5-turbo"):
    system_message = [{"role": "system", "content": system_message}]
    user_message = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=system_message + examples + user_message,
    )
    return response.choices[0].message.content

In [ ]:
print(get_completion(prompt))

# Beispiel: Bewerber-Maschine

In [ ]:
def get_website_text(url: str):
    response = requests.get(url)

    # Überprüfen, ob die Anfrage erfolgreich war
    if response.status_code == 200:
        # Parsen des HTML-Inhalts der Seite
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extrahieren sämtlichen Textes der Webseite
        text = soup.get_text(separator=' ', strip=True)
        return text
    else:
        # Zurückgeben einer Fehlermeldung mit dem HTTP-Statuscode
        return f"Fehler beim Abrufen der Webseite: HTTP-Statuscode {response.status_code}"

In [ ]:
def get_website_urls(url: str):
    response = requests.get(url)

    # Überprüfen, ob die Anfrage erfolgreich war
    if response.status_code == 200:
        # Parsen des HTML-Inhalts der Seite
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extrahieren sämtlichen Textes der Webseite
        hyperlinks = soup.find_all('a')
        return [link.get('href') for link in hyperlinks if link.get('href')]
    else:
        # Zurückgeben einer Fehlermeldung mit dem HTTP-Statuscode
        return f"Fehler beim Abrufen der Webseite: HTTP-Statuscode {response.status_code}"

In [ ]:
def get_link_by_substring(strings: [str], substrings: [str]):
    # Konvertierung der Substrings in Kleinbuchstaben für case-insensitive Suche
    substrings_lower = [substring.lower() for substring in substrings]

    # Filtern der Liste durch Überprüfung, ob irgendein Substring in jedem Element enthalten ist
    filtered_list = [s for s in strings if any(substring_lower in s.lower() for substring_lower in substrings_lower)]

    return filtered_list

In [ ]:
# Hilfsfunktion zur Interaktion mit der Chat-API
def get_completion(messages: [str], model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [ ]:
applicant_profile = """
Name des Bewerbers ist Max Mustermann; abgeschlossenes Abitur mit Note 1,7; Leistungskurse Mathematik und Physik;
Interesse an digitalen Geschäftsmodellen im Handel; Erstes Praktikum in einem Startup, das über einen online-shop schmuck verkauft;
Programmiererfahrung in Python, insbesondere Webprogrammierung; motiviert und zuverlässig
"""

In [ ]:
system_message = f"""
Du bist ein Experte bei der Erstellung professioneller Anschreiben für Bewerbungen! Du erstellst ein Anschreiben für das in einfachen Anführungszeichen beschriebene
Bewerberprofil.

'{applicant_profile}'
"""

In [ ]:
def add_job_description_to_prompt(job_description: str):
    prompt = f"""
    Du erhälst in einfachen Anführungszeichen einen Ausschreibungstext für ein duales Studium. Bitte verwende das Bewerberprofil, um ein Anschreiben für eine Bewerbung
    auf die Stellenausschreibung zu erstellen. Das Anschreiben soll professionell geschrieben sein. Das Anschreiben soll spezifisch auf die Anforderungen in der
    Ausschreibung eingehen und zeigen, dass der Bewerber diese bestmöglich erfüllt. Die Länge des Anschreibens soll eine DIN-A4 Seite sein.

    '{job_description}'
    """
    return prompt

In [ ]:
links = get_website_urls("https://www.lappcareer.com/stellenangebote.html?")

In [ ]:
relevant_links = get_link_by_substring(links,["duales studium","duales-studium", "dual", "dhbw"])
print(f"{len(relevant_links)} passende Stellenausschreibungen gefunden!")

In [ ]:
job_descriptions = []
for link in relevant_links:
    job_descriptions.append(get_website_text(link))

In [ ]:
cover_letter = []

In [ ]:
for job in job_descriptions:
    messages =  [
    {'role':'system',
    'content': system_message},
    {'role':'user',
    'content': add_job_description_to_prompt(job)},
    ]
    cover_letter.append(get_completion(messages))

In [ ]:
print(cover_letter[0])